**After running this section a folder named my_files will open , and you'll see their the model and the explainer file**

In [ ]:
# =======================
# LOAD THE TRAINED MODEL
# =======================

# Install gdown if not already installed
!pip install gdown --upgrade --quiet

# Download the zip file
!gdown 1Q3uEqXi_OTwnqVjVXSFBvwuxKJ-aPfb1 -O inference.zip
!unzip -q inference.zip -d inference


!gdown 1DEBhlGkKPwoFVg-TfqVT6givah6vqT1I -O scalar.zip
!unzip -q scalar.zip -d scalar


!gdown 1rKlHYbQ--54W6vWfn3gMDi83YxcTyOzU -O inference_explainer.zip
!unzip -q inference_explainer.zip -d inference_explainer

!pip install ipython-autotime
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-0fhqH8tXKPb60C_b4aUHT7f-J4O6Ezq
%load_ext autotime


Downloading...
From: https://drive.google.com/uc?id=1Q3uEqXi_OTwnqVjVXSFBvwuxKJ-aPfb1
To: /content/inference.zip
100% 11.8k/11.8k [00:00<00:00, 28.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DEBhlGkKPwoFVg-TfqVT6givah6vqT1I
To: /content/scalar.zip
100% 664/664 [00:00<00:00, 3.24MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rKlHYbQ--54W6vWfn3gMDi83YxcTyOzU
To: /content/inference_explainer.zip
100% 2.09k/2.09k [00:00<00:00, 8.85MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-0fhqH8tXKPb60C_b4aUHT7f-J4O6Ezq
From (redirected): https://drive.google.com/uc?id=1-0fhqH8tXKPb60C_b4aUHT7f-J4O6Ezq&confirm=t&uuid=817d2dba-6892-4038-8b44-a8912e3a4ca6
To: /content/dataset_labels.npz
100% 31.4M/31.4M [00:00<00:00, 100MB/s]
time: 371

**After the model There will be option to run gymnsium for play game **

In [ ]:
# Install required libraries
!pip install gymnasium moviepy==1.0.3 tensorflow==2.17.1 --quiet

# Import necessary libraries
import gymnasium as gym
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import joblib

# Print TensorFlow version for debugging
print(f"Using TensorFlow version: {tf.__version__}")

# Load the pre-trained model
try:
    # Check if the model path exists
    model_path = 'inference/cartpole_imitation_100.h5'
    chosen_model = load_model(model_path)
    chosen_model.compile(optimizer=Adam(learning_rate=0.001),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")



try:
    scaler_path = "scalar/cartpole_scaler.pkl"
    scaler = joblib.load(scaler_path)
    print("Scaler loaded successfully!")
except Exception as e:
    print(f"Error loading scaler: {e}")


Using TensorFlow version: 2.17.1


Model loaded successfully!
Scaler loaded successfully!
time: 10.7 s (started: 2025-02-03 07:59:10 +00:00)


In [ ]:
!pip install gymnasium --quiet

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import ImageSequenceClip
from IPython.display import display, Video

import gymnasium
print(f"Gymnasium version: {gym.__version__}")




Gymnasium version: 1.0.0
time: 3.66 s (started: 2025-02-03 07:59:21 +00:00)


**Playing the games **

In [ ]:

from sklearn.preprocessing import StandardScaler
import numpy as np

# =======================
# FUNCTION TO PLAY GAME USING TRAINED MODEL
# =======================
def play_game(env, chosen_model, scaler, max_steps=500):
    """
    Play the game using the trained chosen_model with normalized input.

    Args:
        env: Gym environment instance.
        chosen_model: Trained TensorFlow/Keras chosen_model.
        scaler: StandardScaler used for normalizing states.
        max_steps: Maximum steps to play in the game.

    Returns:
        frames: List of frames for the game.
        total_reward: Total reward accumulated during the game.
    """
    state, _ = env.reset()
    frames = []
    total_reward = 0

    for _ in range(max_steps):
        # Normalize the state using the loaded scaler
        state_normalized = scaler.transform([state])

        # Predict the action
        action_probs = chosen_model.predict(state_normalized, verbose=0)
        action = np.argmax(action_probs[0])

        # Take action in the environment
        state, reward, done, truncated, _ = env.step(action)
        total_reward += reward

        # Store frame
        frames.append(env.render())

        if done or truncated:
            break

    return frames, total_reward

# =======================
# FUNCTION TO CREATE VIDEO
# =======================
def create_video(frames, output_path='gameplay.mp4', fps=30):
    """
    Create a video from game frames.

    Args:
        frames: List of frames from the game.
        output_path: Path to save the video.
        fps: Frames per second for the video.
    """
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(output_path, codec='libx264', logger=None)
    return output_path

# =======================
# RUN MULTIPLE GAMES AND RECORD VIDEO
# =======================
env = gym.make("CartPole-v1", render_mode="rgb_array")
num_games = 5

for i in range(1, num_games + 1):
    print(f"Playing game {i}...")
    frames, total_reward = play_game(env, chosen_model, scaler)
    video_path = f'gameplay_{i}.mp4'
    create_video(frames, output_path=video_path)
    print(f"Game {i} - Total Reward: {total_reward}")
    display(Video(video_path, embed=True))

env.close()


Playing game 1...
Game 1 - Total Reward: 500.0


Playing game 2...
Game 2 - Total Reward: 500.0


Playing game 3...
Game 3 - Total Reward: 500.0


Playing game 4...
Game 4 - Total Reward: 500.0


Playing game 5...
Game 5 - Total Reward: 500.0


time: 3min 20s (started: 2025-02-03 07:59:24 +00:00)
